In [1]:
import numpy as np
import qcodes as qc
import broadbean as bb
from broadbean.plotting import plotter

from pytopo.awg_sequencing import broadbean as bbtools
from pytopo.awg_sequencing.broadbean import BluePrints, BroadBeanSequence

ramp = bb.PulseAtoms.ramp
sine = bb.PulseAtoms.sine
sine = bb.PulseAtoms.gaussian

In [3]:
class RabiSequence(BroadBeanSequence):
    """
    A sequence that consists of a single gaussian pulse followed by a readout pulse.

    required channels:
        'I' : analog output (qubit drive pulse)
        'ats_trigger' : marker for the alazar
        'ro_pulse' : readout marker 
        'qb_pulse' : qubit marker
    """
    name = 'rabi_sequence'

    def sequence(self, pre_pulse_time=1e-6, pi_pulse_time=10e-6, after_pulse_time=0.01e-6,
                 dur_trigger_alazar = 1e-6, dur_readout = 100e-6, 
                 n_amplitudes=20, range_amplitudes = 1):
        
        t_pulse = pre_pulse_time + pi_pulse_time + after_pulse_time
        sigma = pi_pulse_time/4.0
        
        elements = []
        for A in np.linspace(-range_amplitudes, range_amplitudes, n_amplitudes):
            bps = bbtools.BluePrints(chan_map=self.chan_map, length=cycle_time, sample_rate=self.SR)
            
            bps['I'].insertSegment(0, ramp, (0, 0), dur=pre_pulse_time)
            bps['I'].insertSegment(1, gaussian, (A, sigma, 0, 0), 
                                   name='gaussian_pulse', dur=pi_pulse_time)
            bps['I'].insertSegment(2, ramp, (0, 0), dur=after_pulse_time)
            bps['qb_pulse'] = [(pre_pulse_time, pre_pulse_time+pi_pulse_time)]
            bps['ats_trigger'] = [(t_pulse, t_pulse + dur_trigger_alazar)]
            bps['ro_pulse'] = [(t_pulse, t_pulse+dur_readout)]

            elements.append(bbtools.blueprints2element(bps))
        
        return bbtools.elements2sequence(elements, self.name)
    
    
RabiSequence.chan_map = {
    1 : ['I', 'ats_trigger', 'ro_pulse'],
    2 : [None, 'qb_pulse', None],   
}

RabiSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
    2 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

rabi_seq = RabiSequence(station.awg)
rabi_seq.wait = 'off'
rabi_seq.setup(plot=True, start_awg=False)

NameError: name 'station' is not defined